# Populate `zip_code_dict.pickle`

^^ this is the one and only thing this file should do

In [47]:
from shapely.geometry import Polygon, Point
import networkx as nx
import random
from uszipcode import SearchEngine
import pickle
import fiona
from shapely.geometry import shape
from pyproj import Proj, transform, Transformer

g_pickle = "01_boston-area.gpickle"
g = nx.read_gpickle(g_pickle)


In [48]:
shapes = fiona.open("./mass_zips/zipcodes_nt/ZIPCODES_NT_POLY.shp")
input_proj = shapes.crs.get('init')
output_proj = "epsg:4326"
transformer = Transformer.from_crs(input_proj, output_proj)

polygons = {}
for shape in shapes:
    zip_code = shape.get('properties').get('POSTCODE')    
    points = []
    for coord in shape.get('geometry').get('coordinates')[0]:
        x = coord[0]
        y = coord[1]
        xy = transformer.transform(x,y)
        points.append([xy[1], xy[0]])
    try: 
        polygons[zip_code] = Polygon(points)
    except: print(zip_code)

02360
02790
01450
02535
01742
02568
02554
02649
01938
01844
01826
02667
02050
02539
02536
01930
01463
02777
01585
02748
02332
01966
02481
02655
02420
02043
01852
02740
02559
01529
01905
01945
02630
02571
02066
01969
01951
01879
01701
02563
02739
01821
01050
02668
02653
01778
01760
01952
01982
01915
02719
02642
01923
01929
02713
02738
02186
02169
01803
02633
01504
02718
02025
02673
01862
01950
01008
01835
01944
02148
02472
01854
02675
02155
02045
02191
02543
01380
02453
02125
02127
02670
02534
02744
02129
02026
02532
01908
02791
02641
01965
02210
02647
02116
02215
02462
02467
02114
02163
02671
02467
02152


In [49]:
with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

In [ ]:
g_pickle = "01_boston-area.gpickle"
g = nx.read_gpickle(g_pickle)
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in polygons:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break
            
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [ ]:
len(zip_code_dict)

In [21]:
search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")
zip_codes = [x for x in polygons]

bad_zips = []
polygons = {}
zip_codes_copy = zip_codes.copy()
for z in zip_codes_copy:
            try: polygons[z] = Polygon(search.by_zipcode(z).polygon)
            except: 
                print(z)
                if z not in bad_zips:
                    bad_zips.append(z)
                zip_codes.remove(z)

with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in zip_codes:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break
            
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)            
        

zip_code_dict
        
unprocessed = []
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    if node_id not in zip_code_dict:
        unprocessed.append(node_id)
print("Number still unprocessed: " + str(len(unprocessed)))
print("Add the following to the zip_codes set:")

['01331', '01085', '01370', '01235', '02747', '02769', '01267', '01008', '01002', '01255', '01247', '01223', '01034', '02346', '01366', '01339', '01340', '01007', '01237', '01452', '01368', '01027', '01341', '01562', '01026', '01266', '01259', '01082', '02330', '01516', '01430', '01257', '01364', '01378', '01541', '01473', '01543', '01256', '02770', '01360', '01337', '01010', '02780', '01005', '01475', '01507', '02035', '01740', '01550', '01469', '01468', '01238', '01810', '01098', '02093', '01095', '01240', '01373', '01752', '01571', '01535', '01451', '01545', '01581', '01568', '01068', '02061', '01540', '01876', '01531', '01520', '02048', '01720', '01351', '02542', '01564', '01077', '01886', '01420', '01037', '01569', '02766', '01050', '01073', '01270', '01033', '02324', '01056', '01301', '01566', '01379', '01748', '01462', '01367', '02703', '01845', '01330', '02038', '02359', '01220', '01523', '01453', '01431', '01245', '02631', '01072', '01776', '01035', '01921', '02067', '02767', 

In [45]:
for polygon in polygons:
    if polygons.get(polygon).contains(Point([-71.1191928, 42.4057])):
        print(polygon)

print(search.by_coordinates(42.4057, -71.1191928, returns=1)[0].zipcode)

# print(polygons.get('02143').contains(Point([-71.1191928, 42.4057])))

02144


In [28]:
unprocessed
g.nodes()['1080135386']

{'lat': 42.4057, 'lon': -71.1191928, 'id': '1080135386', 'tags': {}}

In [ ]:
# def get_zip_code_dict():
#     g = nx.read_gpickle("01_boston-area.gpickle")
#     nodes_data_subset = g.nodes().data()
#     search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")
#     len_g = len(g)
#     count = 1
#     with open('zip_code_dict.pickle', 'rb') as handle:
#         zip_code_dict = pickle.load(handle)

#     for node_data in g.nodes().data():
#         node_id = node_data[1]["id"]
#         lon,lat = node_data[1]['lon'], node_data[1]['lat']
#         if node_id not in zip_code_dict:
#             count += 1
#             zip_code_dict[node_id] = search.by_coordinates(lat, lon, returns=1)[0].zipcode
#             print(zip_code_dict[node_id])
#             print(str(len(zip_code_dict)) + "       "+ str(len(zip_code_dict)/len_g) + "        ...slow algo alert")


#         if count % 25 == 0: 
#             print("--> Dumping into zip_code_dict.pickle")
#             with open('zip_code_dict_NEW.pickle', 'wb') as handle:
#                 pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        

#     with open('zip_code_dict_NEW.pickle', 'wb') as handle:
#         pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     return zip_code_dict

In [ ]:
# get_zip_code_dict()